In [ ]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn


def showOptions():
    """
    Show the options
    """
    choices = ["add a flight", "print flight information (by flight_no)", "delete a flight (by flight_no)",
			"select a flight (by source, dest, stop_no = 0)", "select a flight (by source, dest, stop_no = 1)",
			"exit"]
    print("---------------------------")
    print("Please choose following option:")
    print("1. ", choices[0])
    print("2. ", choices[1])
    print("3. ", choices[2])
    print("4. ", choices[3])
    print("5. ", choices[4])
    print("6. ", choices[5])


def listAllFlights(conn):
    """
    List all flights in the database.
    """
    print("All flights in the database now:")
    with conn:
        sql = "SELECT Flight_no FROM FLIGHTS"
        cur = conn.cursor()
        cur.execute(sql)
        result_set = cur.fetchall()
        for result in result_set:
            print(result[0])
        print("Total", len(result_set), "flight(s).")

def printFlightInfo(flight_no, conn):
    """
    Print out the infomation of a flight given a flight_no
    """
    with conn:
        sql = "SELECT * FROM FLIGHTS WHERE Flight_no =?"
        cur = conn.cursor()
        cur.execute(sql, [flight_no])
        result = cur.fetchone()
        if result == None:
            print("No such flights")
        else:
            headers = ["Flight_no", "Depart_Time", "Arrive_Time", "Fare", "Source", "Dest"]
            for i in range(0, 6):
                print(headers[i], ":", result[i])

def printFlightByNo(conn):
    """
    Select out a flight according to the flight_no.
    """
    listAllFlights(conn)
    line = input("Please input the flight_no to print info:")
    line = line.strip()
    printFlightInfo(line, conn)


def addFlight(conn):
    """
    Insert data into database
    A sample input is: CX109, 2019-03-15 13:00:00, 2019-03-15 19:00:00, 2000, HK, Tokyo
    :return: project id
    """
    line = input("Please input the flight_no, depart_time, arrive_time, fare, source, dest: ")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    with conn:
        sql = "INSERT INTO FLIGHTS VALUES(?, ?, ?, ?, ?, ?)"
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1], values[2], int(values[3]), values[4], values[5]))
        return cur.lastrowid


def deleteFlight(conn):
    """
    Delete a flight from database by flight_no
    """
    listAllFlights(conn)
    line = input("Please input the flight_no to delete: ")
    line = line.strip()
    print(line)
    with conn:
        sql = "DELETE FROM FLIGHTS WHERE FLIGHT_NO=?"
        cur = conn.cursor()
        cur.execute(sql, [line])
        conn.commit()
        print("succeed to delete flight ", line)


def selectFlightsInZeroStop(conn):
    """
    Given source and dest, select all the flights can arrive the dest directly. 
    For example, given HK, Tokyo, you may find a flight from HK -> Tokyo 
    Your job is to fill in this function.
    """
    line = input("Please input source, dest:")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    with conn:
        sql = "SELECT Flight_no FROM FLIGHTS WHERE source = ? AND dest = ?"
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1]))
        result_set = cur.fetchall()
        for result in result_set:
            printFlightInfo(result[0], conn)
            print("=================================================")
        print("Total", len(result_set), "choice(s).")


def selectFlightsInOneStop(conn):
    """
    Given source and dest, select all the flights can arrive the dest in one stop. 
    For example, given HK, Tokyo, you may find HK -> Beijing, Beijing-> Tokyo 
    Your job to fill in this function.
    """
    line = input("Please input source, dest:")
    values = line.split(',')
    for i in range(0, len(values)):
        values[i] = values[i].strip()
    with conn:
        sql = "SELECT F1.Flight_no, F2.Flight_no FROM FLIGHTS F1, FLIGHTS F2 WHERE F1.source = ? AND F1.dest = F2.source AND F1.arrive_time < F2.depart_time AND F2.dest =?"
        cur = conn.cursor()
        cur.execute(sql, (values[0], values[1]))
        result_set = cur.fetchall()
        for result in result_set:
            printFlightInfo(result[0], conn)
            print("-------------------------------------------------")
            printFlightInfo(result[1], conn)
            print("=================================================")
        print("Total", len(result_set), "choice(s).")


"""main"""
database = "lab2database.db"
conn = create_connection(database)
while True:
    showOptions()
    choice = input('Please enter your option: ')
    # with conn:
    if choice == '6':
        conn.close()
        break
    elif choice == '1':
        addFlight(conn)
    elif choice == '2':
        printFlightByNo(conn)
    elif choice == '3':
        deleteFlight(conn)
    elif choice == '4':
        selectFlightsInZeroStop(conn)
    elif choice == '5':
        selectFlightsInOneStop(conn)
    else:
        print("This option is not available")
        continue

---------------------------
Please choose following option:
1.  add a flight
2.  print flight information (by flight_no)
3.  delete a flight (by flight_no)
4.  select a flight (by source, dest, stop_no = 0)
5.  select a flight (by source, dest, stop_no = 1)
6.  exit
Please enter your option: 4
Please input source, dest:HK,Tokyo
Flight_no : CX100
Depart_Time : 2019-03-15 12:00:00
Arrive_Time : 2019-03-15 16:00:00
Fare : 2000
Source : HK
Dest : Tokyo
Flight_no : CX109
Depart_Time : 2019-03-15 13:00:00
Arrive_Time : 2019-03-15 19:00:00
Fare : 2000
Source : HK
Dest : Tokyo
Total 2 choice(s).
---------------------------
Please choose following option:
1.  add a flight
2.  print flight information (by flight_no)
3.  delete a flight (by flight_no)
4.  select a flight (by source, dest, stop_no = 0)
5.  select a flight (by source, dest, stop_no = 1)
6.  exit
Please enter your option: 5
Please input source, dest:HK, Tokyo
Flight_no : CX102
Depart_Time : 2019-03-15 10:00:00
Arrive_Time : 2019-03-